In [96]:
import os
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [13]:
data_list = []
folder_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24")
for x in folder_file:
    item_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24/{}".format(x))
    for y in item_file:
        data_list.append("ravdess_dataset/audio_speech_actors_01-24/{}/{}".format(x,y))


In [14]:
len(data_list)

1440

In [19]:
sample_rate_dataframe = pd.DataFrame(columns=["File_Name", "Sample_Rate", "Duration"])
for item in data_list:
    sample_rate = librosa.get_samplerate(item)
    duration = librosa.get_duration(path=item, sr=sample_rate)
    new_row = {"File_Name": item,"Sample_Rate": sample_rate, "Duration": duration}
    sample_rate_dataframe = pd.concat(
            [sample_rate_dataframe, pd.DataFrame(new_row, index=[0])])
sample_rate_dataframe = sample_rate_dataframe.reset_index(drop=True)

In [50]:
print(sample_rate_dataframe.Duration.min())
print(sample_rate_dataframe.Duration.max())
print(sample_rate_dataframe.Duration.mean())
print(sample_rate_dataframe.Sample_Rate.value_counts())


2.9362708333333334
5.2719375
3.7006648148148145
48000    1440
Name: Sample_Rate, dtype: int64


In [56]:
labels = {'disgust': 0, 'happy': 1, 'sad': 2,
            'neutral': 3, 'fear': 4, 'angry': 5}
data_sentiment_path = []
data_sentiment_value = []
data_sentiment_encoded_value = []
for file in data_list:
    sentiment_code = file[-18:-16]
    if sentiment_code == '05':
        data_sentiment_path.append(file)
        data_sentiment_value.append('angry')  
        data_sentiment_encoded_value.append(5)        
    elif sentiment_code == '07':
        data_sentiment_path.append(file)
        data_sentiment_value.append('disgust')  
        data_sentiment_encoded_value.append(0)
    elif sentiment_code == '06':
        data_sentiment_path.append(file)
        data_sentiment_value.append('fear')  
        data_sentiment_encoded_value.append(4)
    elif sentiment_code == '03':
        data_sentiment_path.append(file)
        data_sentiment_value.append('happy')  
        data_sentiment_encoded_value.append(1)
    elif sentiment_code == '01':
        data_sentiment_path.append(file)
        data_sentiment_value.append('neutral')  
        data_sentiment_encoded_value.append(3)
    elif sentiment_code == '04':
        data_sentiment_path.append(file)
        data_sentiment_value.append('sad')  
        data_sentiment_encoded_value.append(2)
    else:
        pass

In [77]:
num_mfcc = 40
SAMPLE_RATE = 48000
n_fft = 2048
hop_length = 512

train_data = {
    "labels": [],
    "mfcc": []
}

for path, value in zip(data_sentiment_path, data_sentiment_encoded_value):
    signal, sample_rate = librosa.load(path)
    mfcc = librosa.feature.mfcc(
        y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T

    
    train_data['labels'].append(value)
    train_data["mfcc"].append(np.asarray(mfcc))

processed_data_value = np.asarray(train_data['mfcc'])
processed_data_target = np.asarray(train_data["labels"])
    

C:\Users\USER\AppData\Local\Temp\ipykernel_16024\2681898832.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  processed_data_value = np.asarray(train_data['mfcc'])


In [64]:
def vectorToMFCC(vector):
  vector.T
  librosa.display.specshow(vector, hop_length=512)
  plt.xlabel("Time")
  plt.ylabel("MFCC")
  plt.colorbar()
  plt.show()


In [107]:
processed_data_value

IndexError: tuple index out of range

In [110]:
shape_processed = []
for x in processed_data_value:
    shape_processed.append(x.shape[0])

In [115]:
from statistics import mean
mean(shape_processed)

160.82291666666666

In [99]:
padded_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    processed_data_value)

In [100]:
padded_data_value.shape

(1056, 228, 40)

In [116]:
np.save("ravdess_data-value", padded_data_value)
np.save("ravdess_data_target", processed_data_target)

In [120]:
from sklearn.model_selection import train_test_split

train_data_value, test_data_value, train_data_target, test_data_target = train_test_split(padded_data_value, processed_data_target, test_size=0.2, random_state=42)


Shape of X_train: (844, 228, 40)
Shape of X_test: (212, 228, 40)


In [121]:
train_data_value

array([[[   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        ...,
        [-669,    0,    0, ...,    0,    0,    0],
        [-669,    0,    0, ...,    0,    0,    0],
        [-669,    0,    0, ...,    0,    0,    0]],

       [[   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        ...,
        [-877,   13,   13, ...,    0,    0,    0],
        [-879,   10,   10, ...,    0,    0,    0],
        [-876,   14,   13, ...,    0,    0,    0]],

       [[   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        ...,
        [-660,    0,    0, ...,    0,    0,    0],
        [-659,    1,    1, ...,    1,    1,    1],
        [-659,    1,    1, ...,    0,    0,    0]],

       ...,

       [[   0,    0,    

In [122]:
train_data_target

array([1, 3, 1, 5, 2, 4, 1, 0, 5, 5, 5, 2, 4, 4, 2, 1, 0, 2, 2, 2, 3, 1,
       4, 2, 0, 4, 1, 3, 0, 5, 1, 2, 3, 1, 1, 2, 0, 1, 3, 1, 5, 1, 0, 1,
       5, 3, 5, 2, 4, 0, 4, 1, 4, 1, 2, 5, 1, 1, 4, 4, 4, 4, 5, 4, 5, 2,
       3, 1, 5, 0, 5, 4, 0, 4, 3, 2, 2, 5, 1, 5, 2, 4, 2, 0, 5, 1, 4, 0,
       0, 4, 4, 2, 4, 4, 0, 3, 0, 4, 2, 3, 0, 0, 0, 4, 1, 3, 1, 2, 1, 5,
       1, 1, 0, 5, 2, 3, 5, 0, 2, 5, 0, 1, 4, 1, 4, 2, 5, 1, 1, 1, 5, 3,
       0, 2, 1, 1, 2, 0, 4, 4, 0, 0, 1, 1, 5, 4, 3, 1, 0, 3, 1, 4, 2, 1,
       1, 5, 1, 2, 3, 5, 2, 1, 2, 2, 4, 1, 1, 2, 1, 1, 0, 1, 5, 5, 5, 4,
       1, 0, 1, 0, 0, 4, 2, 0, 2, 2, 5, 5, 0, 1, 3, 1, 3, 4, 3, 0, 0, 5,
       3, 1, 4, 1, 4, 5, 1, 2, 1, 0, 0, 0, 0, 4, 4, 2, 3, 1, 1, 1, 1, 2,
       5, 5, 5, 0, 0, 4, 2, 2, 5, 2, 5, 3, 3, 5, 4, 4, 0, 2, 5, 4, 0, 5,
       0, 2, 1, 0, 1, 5, 2, 1, 4, 4, 0, 5, 2, 2, 2, 2, 3, 0, 2, 3, 0, 2,
       1, 5, 1, 0, 1, 5, 5, 2, 4, 2, 0, 0, 4, 2, 0, 5, 2, 1, 2, 2, 1, 0,
       4, 1, 4, 0, 4, 3, 0, 3, 2, 5, 0, 1, 0, 3, 3,

In [128]:
def cnn_lstm(optimizer='adam', learning_rate=0.0001):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
        padded_data_value.shape[1], padded_data_value.shape[2], 1)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(64, (3, 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)))
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.get(optimizer)
    optimiser.learning_rate.assign(learning_rate)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    # model.summary()
    str_learning_rate = str(learning_rate).replace('.', '')
    # csv_logger = tf.keras.callbacks.CSVLogger(
    #     'robust_cnn_lstm{0}_{1}.csv'.format(optimizer, str_learning_rate))
    history = model.fit(train_data_value, train_data_target, validation_data=(
        test_data_value, test_data_target), batch_size=32, epochs=1)
    return model

In [129]:
cnn_lstm()

27/27 [==============================] - 15s 121ms/step - loss: 1.6856 - accuracy: 0.2903 - val_loss: 1.7427 - val_accuracy: 0.2170


In [130]:
np.save("ravdess_train_data_value", train_data_value)
np.save("ravdess_train_data_target", train_data_target)
np.save("ravdess_test_data_value", test_data_value)
np.save("ravdess_test_data_target", test_data_target)